# TripletDecoder = Inverse of TripletEncoder

In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.
%config InlineBackend.figure_format = 'svg'
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
from pandas.testing import assert_frame_equal

from tsdm.datasets import MIMIC_III, Electricity
from tsdm.encoders import TripletDecoder, TripletEncoder

## Test on MIMIC

In [ ]:
ds = MIMIC_III()
ts = ds.observations.set_index(["UNIQUE_ID", "TIME_STAMP"])

In [ ]:
decoder = TripletDecoder(value_name="VALUENORM")
decoder.fit(ts)
decoded = decoder.encode(ts)

In [ ]:
restored = decoder.decode(decoded)
restored = restored.set_index("LABEL_CODE", append=True).sort_index(
    level=["UNIQUE_ID", "TIME_STAMP", "LABEL_CODE"]
)
original = ts.set_index("LABEL_CODE", append=True).sort_index(
    level=["UNIQUE_ID", "TIME_STAMP", "LABEL_CODE"]
)
assert_frame_equal(original, restored)

## Test on Electricity

In [ ]:
ds = Electricity()
ts = ds.dataset

### Sparse

In [ ]:
encoder = TripletEncoder(sparse=True)
encoder.fit(ts)
encoded = encoder.encode(ts[:100])

In [ ]:
decoder = TripletDecoder(sparse=True, value_name="value")
decoder.fit(encoded)
decoded = decoder.encode(encoded)

In [ ]:
restored = decoder.decode(decoded)
assert_frame_equal(encoded, restored)

## Non-Sparse

In [ ]:
encoder = TripletEncoder(sparse=False)
encoder.fit(ts)
encoded = encoder.encode(ts[:100])

In [ ]:
decoder = TripletDecoder(sparse=False, value_name="value")
decoder.fit(encoded)
decoded = decoder.encode(encoded)

In [ ]:
restored = decoder.decode(decoded)
assert_frame_equal(encoded, restored)